# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [111]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [112]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    ans = [0 for i in range(n)]
    cur = 0
    ans[0] = cur
    used = [False for i in range(n)]
    for step in range(n - 1):
        best = -1
        bestdist = 0
        for vertex in range(n):
            if not used[vertex]:
                dist = euclidean_distance(instance[cur], instance[vertex])
                if best == -1 or bestdist > dist:
                    best = vertex
                    bestdist = dist
        ans[step] = best
        used[best] = True
        cur = best
    return ans

In [113]:
def solve_tsp_nearest_insertion(instance):
    n = len(instance)
    ans = [0]
    used = [False for i in range(n)]
    used[0] = True
    bestcurs = [euclidean_distance(instance[0], instance[i]) for i in range(n)]
    for step in range(n-1):
        best = -1
        bestdist = 0
        for vertex in range(n):
            if not used[vertex]:
                if best == -1 or bestcurs[vertex] < bestdist:
                    best = vertex
                    bestdist = bestcurs[vertex]
        bestarch = 0
        bestplace = -1
        for (index, beg) in enumerate(ans):
            end = ans[(index + 1) % len(ans)]
            d = euclidean_distance(instance[best], instance[beg]) + euclidean_distance(instance[best], instance[end]) - euclidean_distance(instance[beg], instance[end])
            if bestplace == -1 or bestarch > d:
                bestarch = d
                bestplace = index + 1
        ans.insert(bestplace, best)
        used[best] = True
        for vert in range(n):
            if not used[vert]:
                bestcurs[vert] = min(bestcurs[vert], euclidean_distance(instance[best], instance[vert]))
    return ans

In [114]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [115]:
run_all()

Solving instance d198.tsp… done in 0.037 seconds with tour length 18570 using NN and in 0.11 seconds with tour length 18052 using NI
Solving instance d493.tsp… done in 0.14 seconds with tour length 43630 using NN and in 0.61 seconds with tour length 41576 using NI
Solving instance d657.tsp… done in 0.25 seconds with tour length 62076 using NN and in 1.1 seconds with tour length 60195 using NI
Solving instance d2103.tsp… done in 2.6 seconds with tour length 87323 using NN and in 1.1e+01 seconds with tour length 86027 using NI
Solving instance pr107.tsp… done in 0.0064 seconds with tour length 46480 using NN and in 0.027 seconds with tour length 53211 using NI
Solving instance pr152.tsp… done in 0.012 seconds with tour length 85567 using NN and in 0.054 seconds with tour length 86914 using NI
Solving instance pr439.tsp… done in 0.1 seconds with tour length 131056 using NN and in 0.46 seconds with tour length 132780 using NI


## Выводы
Найденные значения не очень сильно отличаются. Иногда один алгоритм работает лучше, иногда- второй.